In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
SPY_data = yf.download(tickers = '^GSPC', start = '2020-01-01', interval = '60m')
SPY_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [4]:
VOO_data = yf.download(tickers = 'VOO', start = '2020-01-01', interval = '60m')
VOO_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [5]:
SPLG_data = yf.download(tickers = 'SPLG', start = '2020-01-01', interval = '60m')
SPLG_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

In [6]:
IVV_data = yf.download(tickers = 'IVV', start = '2020-01-01', interval = '60m')
IVV_data['Adj Close'].isna().sum()

[*********************100%***********************]  1 of 1 completed


0

# Simple base model

In [7]:
def checkIncreasing(df, n):
    for i in range(n - 1):
        if df.iloc[i] > df.iloc[i + 1]:
            return False
        
    return True

In [8]:
def checkDecreasing(df, n):
    for i in range(n - 1):
        if df.iloc[i] < df.iloc[i + 1]:
            return False
        
    return True

In [9]:
def checkPositive(df, n):
    positive = df.iloc[0] >= 0
    if positive:
        for i in range(1, n):
            if df[i] < 0:
                return False
    return True

In [10]:
def checkNegative(df, n):
    negative = df.iloc[0] < 0
    if negative:
        for i in range(1, n):
            if df[i] > 0:
                return False
            
    return True

In [11]:
stock_df = pd.DataFrame(columns = ['SPY', 'VOO', 'SPLG', 'IVV'], index = SPY_data['2020-05-01':].index[4:])
totalTransactionCost = 0

In [46]:
def calculate_base_everything(asset_data, column_name, stock_df, past_shift, check_range):
    asset_Moment = (asset_data['Adj Close']['2020-05-01':] - asset_data['Adj Close'].shift(past_shift)).dropna()
    asset_MAVG = asset_data['Adj Close']['2020-05-01':].rolling(past_shift, min_periods = 1).mean().dropna()
    margins = 100000000 / 4
    currAmount = 100000000 / 4
    holdings = 0
    actions = []
    global totalTransactionCost
    for i, date in enumerate(asset_Moment['2020-05-01':].index[check_range:]):
        currAmount = holdings * asset_data['Adj Close'][date] + margins
        actions.append('hold')
        if holdings == 0:
            if checkIncreasing(asset_MAVG.iloc[i: i + check_range], check_range):
                if checkPositive(asset_Moment[i: i + check_range], check_range):
                    if margins >= currAmount * 0.02:
                        holdings += currAmount * 0.02 / (asset_data['Adj Close'][date] * 1.02)
                        margins -= currAmount * 0.02 * 1.01
                        totalTransactionCost += currAmount * 0.02 * 0.01
            if checkDecreasing(asset_MAVG.iloc[i: i + check_range], check_range):
                if checkNegative(asset_Moment[i: i + check_range], check_range):
                    holdings -= currAmount * 0.02 / (asset_data['Adj Close'][date] * 0.98)
                    margins += currAmount * 0.02 * 0.99
                    totalTransactionCost += currAmount * 0.02 * 0.01
        else:
            if holdings < 0:
                if checkIncreasing(asset_MAVG.iloc[i: i + check_range], check_range):
                    if checkPositive(asset_Moment[i: i + check_range], check_range):
                        margins -= holdings * (asset_data['Adj Close'][date] * 1.02) * 1.01
                        holdings = 0
                        totalTransactionCost += currAmount * 0.02 * 0.01
            elif holdings > 0:
                if checkDecreasing(asset_MAVG.iloc[i: i + check_range], check_range):
                    if checkNegative(asset_Moment[i: i + check_range], check_range):
                        margins += holdings * (asset_data['Adj Close'][date] * 0.98) * 0.99
                        holdings = 0
                        totalTransactionCost += currAmount * 0.02 * 0.01
        asset_currAmount = holdings * asset_data['Adj Close'][date] + margins
        stock_df.loc[date, column_name] = currAmount

In [312]:
stat_df = pd.DataFrame(index = np.arange(1, 20), columns = np.arange(1, 20))
for i in range(1, 20):
    for j in range(1, 20):
        stock_df = pd.DataFrame(columns = ['SPY', 'VOO', 'SPLG', 'IVV'], index = SPY_data['2020-05-01':].index[j:])
        totalTransactionCost = 0
        calculate_base_everything(SPY_data, 'SPY', stock_df, i, j)
        calculate_base_everything(VOO_data, 'VOO', stock_df, i, j)
        calculate_base_everything(SPLG_data, 'SPLG', stock_df, i, j)
        calculate_base_everything(IVV_data, 'IVV', stock_df, i, j)
        net_profit = stock_df.iloc[-1].sum() - 100000000
        net_return = net_profit / 100000000
        stat_df.loc[i, j] = net_profit

In [313]:
stat_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,81557856.403931,2120344193554426.5,59093198578.709946,1520217669.601561,222861234.606357,64812128.432167,23300420.016254,16228683.115246,11111291.136402,-72589.72591,125940.524605,28476.578208,35715.89539,0.0,0.0,0.0,0.0,0.0,0.0
2,81557856.403931,56103549937648.859375,193727354324.603912,5560241388.126545,706836462.836475,162917420.37372,91103127.645886,41429186.888428,27093950.891633,4313494.137089,5225499.571322,995526.90656,624392.84163,582578.756908,584070.984619,560324.642043,537828.229164,539015.640444,34654.906932
3,81557856.403931,6902873915595.012695,100114883294.138184,9655052238.62562,1697576419.450742,432321236.888451,218701836.506003,111053524.377045,54097767.543176,32284947.511329,16503895.563356,12119389.931419,9048384.723238,4969668.00421,5101441.763223,707383.817366,704828.721554,702588.896706,715696.215019
4,81557856.403931,674363155934.616577,48957690869.410904,13995968515.574387,2854431358.328905,834688123.547109,299682936.859331,187729847.077021,72010151.136039,34169517.441839,29767181.025666,22497209.804269,17744468.6024,13359685.387704,13456974.7983,8745997.572507,6621936.212215,4510557.836946,334314.960379
5,81557856.403931,314704635022.105957,29882305930.785912,8834746752.159254,3758110114.885915,1288722200.713843,614136755.350347,251060941.112979,105137821.56935,49387718.424579,42988998.08354,37110670.403068,32767585.085048,22438104.548208,22758401.884574,22739430.501093,21459552.434767,8452443.855531,8847005.318217
6,81557856.403931,146449966675.88736,19873094565.3764,7125113122.325643,3381568606.855268,2075684263.796276,770356453.902789,292407677.399633,130668624.4226,93637954.677648,71836342.146312,56074926.995867,44193717.453349,33782800.28212,21266197.906209,18006214.788247,13426632.344341,8981669.885151,8908308.885852
7,81557856.403931,93639475513.666702,12490580758.16967,5209005021.862885,2966556746.100496,1498585927.076931,1050916766.917539,340639686.812043,262429898.097026,174161675.842889,108385851.268349,85875627.135595,58821640.775119,42353103.869075,34887897.451333,27559440.136248,22367738.840572,17624794.21972,17730072.099708
8,81557856.403931,53083626139.150436,8643079362.28236,3555060534.181812,2224649371.392803,1203192408.563857,840452746.105034,442814290.851884,307798027.03686,211533595.078847,127285649.868709,92398118.001002,68253664.30995,62377553.427734,42466594.434268,42886534.913302,33208651.091903,32305682.451862,27335024.23102
9,81557856.403931,40782364402.126884,6938744766.887856,2543344162.919222,1601608434.803,974721319.110433,744913700.776625,476297013.196933,358132149.507366,251750336.050943,171016208.114281,126098713.402728,83780645.772545,72012031.934103,63723411.36213,51048155.320214,44083458.300231,27777032.85079,22188646.638204
10,81557856.403931,33838909972.632759,4155735232.046383,2031003747.257796,981477195.811549,674003192.802343,551031792.257601,414618991.455126,303372246.619822,286176051.289451,204716732.088019,159237081.617849,105994241.118896,107162536.543942,71511167.622742,65480902.777795,58690754.216392,41265634.979682,29518225.847278


In [314]:
(stat_df - 100000000) / 100000000

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,-0.184421,21203440.935544,589.931986,14.202177,1.228612,-0.351879,-0.766996,-0.837713,-0.888887,-1.000726,-0.998741,-0.999715,-0.999643,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-0.184421,561034.499376,1936.273543,54.602414,6.068365,0.629174,-0.088969,-0.585708,-0.72906,-0.956865,-0.947745,-0.990045,-0.993756,-0.994174,-0.994159,-0.994397,-0.994622,-0.99461,-0.999653
3,-0.184421,69027.739156,1000.148833,95.550522,15.975764,3.323212,1.187018,0.110535,-0.459022,-0.677151,-0.834961,-0.878806,-0.909516,-0.950303,-0.948986,-0.992926,-0.992952,-0.992974,-0.992843
4,-0.184421,6742.631559,488.576909,138.959685,27.544314,7.346881,1.996829,0.877298,-0.279898,-0.658305,-0.702328,-0.775028,-0.822555,-0.866403,-0.86543,-0.91254,-0.933781,-0.954894,-0.996657
5,-0.184421,3146.04635,297.823059,87.347468,36.581101,11.887222,5.141368,1.510609,0.051378,-0.506123,-0.57011,-0.628893,-0.672324,-0.775619,-0.772416,-0.772606,-0.785404,-0.915476,-0.91153
6,-0.184421,1463.499667,197.730946,70.251131,32.815686,19.756843,6.703565,1.924077,0.306686,-0.06362,-0.281637,-0.439251,-0.558063,-0.662172,-0.787338,-0.819938,-0.865734,-0.910183,-0.910917
7,-0.184421,935.394755,123.905808,51.09005,28.665567,13.985859,9.509168,2.406397,1.624299,0.741617,0.083859,-0.141244,-0.411784,-0.576469,-0.651121,-0.724406,-0.776323,-0.823752,-0.822699
8,-0.184421,529.836261,85.430794,34.550605,21.246494,11.031924,7.404527,3.428143,2.07798,1.115336,0.272856,-0.076019,-0.317463,-0.376224,-0.575334,-0.571135,-0.667913,-0.676943,-0.72665
9,-0.184421,406.823644,68.387448,24.433442,15.016084,8.747213,6.449137,3.76297,2.581321,1.517503,0.710162,0.260987,-0.162194,-0.27988,-0.362766,-0.489518,-0.559165,-0.72223,-0.778114
10,-0.184421,337.3891,40.557352,19.310037,8.814772,5.740032,4.510318,3.14619,2.033722,1.861761,1.047167,0.592371,0.059942,0.071625,-0.284888,-0.345191,-0.413092,-0.587344,-0.704818


In [47]:
stock_df = pd.DataFrame(columns = ['SPY', 'VOO', 'SPLG', 'IVV'], index = SPY_data['2020-05-01':].index[4:])
totalTransactionCost = 0
calculate_base_everything(SPY_data, 'SPY', stock_df, 4, 4)
calculate_base_everything(VOO_data, 'VOO', stock_df, 4, 4)
calculate_base_everything(SPLG_data, 'SPLG', stock_df, 4, 4)
calculate_base_everything(IVV_data, 'IVV', stock_df, 4, 4)

In [48]:
display(stock_df)

,SPY,VOO,SPLG,IVV
Datetime,,,,
2020-05-01 13:30:00-04:00,25000000.0,25000000.0,25000000.0,25000000.0
2020-05-01 14:30:00-04:00,24984749.090387,24984772.366031,24984642.268237,24984715.001455
2020-05-01 15:30:00-04:00,24984893.135893,24984752.769048,24984949.627121,24984795.918367
2020-05-04 09:30:00-04:00,24987535.245297,24987462.726078,24987408.439571,24987566.142363
2020-05-04 10:30:00-04:00,24987900.855304,24987729.832349,24987715.798455,24987934.879746
...,...,...,...,...
2021-12-03 11:30:00-05:00,3485522421.524962,3786786453.566146,3344849109.822896,3496342009.107561
2021-12-03 12:30:00-05:00,3483505823.156521,3784597451.160047,3342915253.541872,3494320622.025889
2021-12-03 13:30:00-05:00,3481584397.099092,3782509517.247171,3341074678.295687,3492399774.212921


In [49]:
net_profit = stock_df.iloc[-1].sum() - 100000000
net_return = net_profit / 100000000
net_profit, net_return

(13995909496.290842, 139.9590949629084)

In [50]:
sum_port = stock_df.sum(axis = 1)

In [51]:
sum_port.std()

3358848935.592488

In [39]:
hourly_return = ((sum_port - (sum_port.shift(1))) / sum_port).dropna()
hourly_return

Datetime
2020-05-01 14:30:00-04:00   -0.000612
2020-05-01 15:30:00-04:00    0.000005
2020-05-04 09:30:00-04:00    0.000106
2020-05-04 10:30:00-04:00    0.000013
2020-05-04 11:30:00-04:00   -0.000061
                               ...   
2021-12-03 11:30:00-05:00   -0.000037
2021-12-03 12:30:00-05:00   -0.000579
2021-12-03 13:30:00-05:00   -0.000551
2021-12-03 14:30:00-05:00   -0.000014
2021-12-03 15:30:00-05:00   -0.000104
Length: 2807, dtype: float64

In [40]:
hourly_return.std() * np.sqrt(252 * 7)

0.33763755300049697

In [41]:
net_profit / sum_port.std()

4.166876738034073

In [42]:
risk_return = (net_return * 4 - 0.012) / (hourly_return.std() * np.sqrt(252 * 7))
risk_return

1658.0631356809108

In [43]:
(net_return - 0.012 / 4) / hourly_return.std() / np.sqrt(63 * 7)

829.0315678404555

In [44]:
vol = hourly_return.std() * np.sqrt(252 * 7)
vol

0.33763755300049697

In [45]:
SPY_hourly = ((SPY_data - SPY_data.shift(1)) / SPY_data).dropna()
SPY_hourly

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2020-01-02 10:30:00-05:00,-0.002324,-0.002007,0.000056,0.000651,0.000651,1.000000
2020-01-02 11:30:00-05:00,0.000626,-0.000278,-0.000374,-0.000083,-0.000083,-0.221765
2020-01-02 12:30:00-05:00,-0.000099,0.000462,0.001435,-0.000241,-0.000241,-0.155298
2020-01-02 13:30:00-05:00,-0.000222,0.000496,-0.000191,0.001454,0.001454,0.107487
2020-01-02 14:30:00-05:00,0.001442,0.001049,0.001630,0.001022,0.001022,0.172749
...,...,...,...,...,...,...
2021-12-03 11:30:00-05:00,-0.005526,-0.004625,-0.002359,-0.001908,-0.001908,-0.186724
2021-12-03 12:30:00-05:00,-0.001886,-0.000760,0.002378,-0.000349,-0.000349,-0.259105
2021-12-03 13:30:00-05:00,-0.000407,-0.002430,-0.003595,-0.002780,-0.002780,0.120060


In [25]:
rf = 0.012 / 252 / 7

In [26]:
a = SPY_hourly['Adj Close'][hourly_return.index]

In [27]:
na_indexes = a[pd.isna(a)].index

In [28]:
model = sm.OLS(hourly_return - rf, SPY_hourly['Adj Close'][hourly_return.index] - rf).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):             -0.000
Method:                 Least Squares   F-statistic:                             0.1293
Date:                Sat, 04 Dec 2021   Prob (F-statistic):                       0.719
Time:                        10:18:53   Log-Likelihood:                          9494.1
No. Observations:                2807   AIC:                                 -1.899e+04
Df Residuals:                    2806   BIC:                                 -1.898e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Adj Close      0.0146      0.041      0.360      0.719      -0.065       0.094
==============================================================================
Omnibus:                     2436.919   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44043.321
Skew:                           4.357   Prob(JB):                         0.00
Kurtosis:                      20.339   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
tr = (net_return - 0.012 / 4) / model.params[0]
tr

9570.08685579227

In [30]:
stock_df.iloc[-1] / stock_df.iloc[-1].sum()

SPY     0.246965
VOO     0.268309
SPLG    0.236996
IVV      0.24773
Name: 2021-12-03 15:30:00-05:00, dtype: object

In [31]:
totalTransactionCost

291754408.24975395

# ML Models

In [198]:
def compute_WPR(asset_data, lookback):
    highHn = asset_data['High'].rolling(lookback).max()
    lowLn = asset_data['Low'].rolling(lookback).min()
    close = asset_data['Adj Close']
    wpr = -100 * ((highHn - close) / (highHn - lowLn))
    return wpr

In [199]:
def compute_EMA(asset_data, hours, smoothing):
    result = pd.DataFrame(index = asset_data.index[hours:], columns = ['EMA'])
    for i in range(result.shape[0]):
        if i == 0:
            result['EMA'].iloc[i] = asset_data['Adj Close'].iloc[i]
        elif i < hours:
            result['EMA'].iloc[i] = (result['EMA'].iloc[i - 1] * i + asset_data['Adj Close'].iloc[i]) / (i + 1)
        else:
            result['EMA'].iloc[i] = asset_data['Adj Close'].iloc[i] * (1 + smoothing) / (1 + hours) \
                + result['EMA'].iloc[i - 1] * (1 - (smoothing / (1 + hours)))
    return result

In [200]:
def calculate_model_indicators(asset_data, wpr_look_back, hours, smoothing):
    asset_Moment = (asset_data['Adj Close']['2020-05-01':] - asset_data['Adj Close'].shift(6)).dropna()
    asset_MAVG = asset_data['Adj Close']['2020-05-01':].rolling(6, min_periods = 1).mean().dropna()
    asset_WPR = compute_WPR(asset_data['2020-04-28':], wpr_look_back)
    asset_EMA = compute_EMA(asset_data['2020-04-28':], hours, smoothing)
    return asset_Moment, asset_MAVG, asset_WPR, asset_EMA

In [201]:
moment, mavg, wpr, ema = calculate_model_indicators(SPY_data, 15, 21, 3)

In [202]:
SPY_df = pd.concat((moment.rename('moment'), mavg.rename('mavg'), wpr.rename('wpr'), ema), axis = 1).shift(1).dropna()
SPY_df['label'] = None
SPY_df

,moment,mavg,wpr,EMA,label
Datetime,,,,,
2020-05-01 10:30:00-04:00,-59.180176,2855.679932,-92.995818,2893.98999,None
2020-05-01 11:30:00-04:00,-52.780029,2850.329956,-99.918080,2886.599976,None
2020-05-01 12:30:00-04:00,-74.800049,2843.386637,-96.817956,2881.159993,None
2020-05-01 13:30:00-04:00,-76.419922,2839.347473,-95.991223,2880.942505,None
2020-05-01 14:30:00-04:00,-78.850098,2838.053955,-91.542382,2882.247998,None
...,...,...,...,...,...
2021-12-03 11:30:00-05:00,-50.549805,4569.603271,-78.577688,6193.342114,None
2021-12-03 12:30:00-05:00,-43.479980,4562.356608,-84.214564,6182.071829,None
2021-12-03 13:30:00-05:00,-65.100098,4551.506592,-83.611353,6171.405695,None


In [203]:
for date in SPY_df.index:
    SPY_df.loc[date, 'label'] = -1 if SPY_data.loc[date]['Open'] >  SPY_data.loc[date]['Close'] else 1

In [204]:
SPY_df

,moment,mavg,wpr,EMA,label
Datetime,,,,,
2020-05-01 10:30:00-04:00,-59.180176,2855.679932,-92.995818,2893.98999,-1
2020-05-01 11:30:00-04:00,-52.780029,2850.329956,-99.918080,2886.599976,-1
2020-05-01 12:30:00-04:00,-74.800049,2843.386637,-96.817956,2881.159993,-1
2020-05-01 13:30:00-04:00,-76.419922,2839.347473,-95.991223,2880.942505,1
2020-05-01 14:30:00-04:00,-78.850098,2838.053955,-91.542382,2882.247998,1
...,...,...,...,...,...
2021-12-03 11:30:00-05:00,-50.549805,4569.603271,-78.577688,6193.342114,-1
2021-12-03 12:30:00-05:00,-43.479980,4562.356608,-84.214564,6182.071829,-1
2021-12-03 13:30:00-05:00,-65.100098,4551.506592,-83.611353,6171.405695,-1


In [212]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

In [213]:
X_train, X_test, y_train, y_test = train_test_split(SPY_df.iloc[:, :-1], SPY_df.iloc[:, -1], test_size=0.25, random_state=42)

In [214]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [215]:
for i in range(7, 15):
    for j in range(5, 10):
        rf = RandomForestClassifier(max_depth = i, min_samples_leaf = j)
        rf.fit(X_train, y_train)
        y_test_predict = rf.predict(X_test)
        print(accuracy_score(np.asarray(y_test), y_test_predict), accuracy_score(y_train, np.asarray(rf.predict(X_train))), i)

0.519203413940256 0.6864326375711575 7
0.5334281650071123 0.6864326375711575 7
0.5163584637268848 0.6778937381404174 7
0.5234708392603129 0.6731499051233396 7
0.5248933143669986 0.6574952561669829 7
0.5248933143669986 0.7319734345351043 8
0.5163584637268848 0.7205882352941176 8
0.5106685633001422 0.7125237191650854 8
0.5206258890469416 0.6888045540796964 8
0.5376955903271693 0.7096774193548387 8
0.5177809388335705 0.7741935483870968 9
0.5149359886201992 0.7471537001897534 9
0.5234708392603129 0.7466793168880456 9
0.5064011379800853 0.7324478178368121 9
0.5064011379800853 0.7367172675521821 9
0.5135135135135135 0.7813092979127134 10
0.48933143669985774 0.7817836812144212 10
0.5149359886201992 0.7751423149905123 10
0.5035561877667141 0.7670777988614801 10
0.5049786628733998 0.7675521821631879 10
0.5206258890469416 0.8121442125237192 11
0.5064011379800853 0.821157495256167 11
0.5092460881934566 0.790796963946869 11
0.5177809388335705 0.7784629981024668 11
0.5049786628733998 0.771347248576

In [220]:
dc_tree = RandomForestClassifier(max_depth = 14, min_samples_leaf = 7)
dc_tree.fit(X_train, y_train)

RandomForestClassifier(max_depth=14, min_samples_leaf=7)

In [231]:
predicted = dc_tree.predict(SPY_df.iloc[:, :-1])
currAmount = 25000000
for i in range(predicted.size):
    if predicted[i] == 1:
        shares = currAmount * 0.02 * 0.99 / (SPY_data.loc[SPY_df.index[i]]['Open'] * 1.01)
        currAmount -= currAmount * 0.02 * 0.99
        currAmount += shares * (SPY_data.loc[SPY_df.index[i]]['Close'] * 0.99) * 0.99
    else:
        shares = currAmount * 0.02 * 0.99 / (SPY_data.loc[SPY_df.index[i]]['Open'] * 0.99)
        currAmount += currAmount * 0.02 * 0.99
        currAmount -= shares * (SPY_data.loc[SPY_df.index[i]]['Close'] * 1.01) * 1.01

In [232]:
currAmount

5000822.004423508

In [216]:
neural_clf = MLPClassifier(hidden_layer_sizes = (145,), activation = 'tanh', alpha = 0.002)
neural_clf.fit(X_train, y_train)
y_test_predict = neural_clf.predict(X_test)
print(accuracy_score(np.asarray(y_test), y_test_predict), accuracy_score(y_train, np.asarray(neural_clf.predict(X_train))))

0.5320056899004267 0.5327324478178368


In [217]:
svc_clf = LinearSVC(max_iter = 3000, penalty = 'l1', C = 10, dual = False)
svc_clf.fit(X_train, y_train)
y_test_predict = svc_clf.predict(X_test)
print(accuracy_score(np.asarray(y_test), y_test_predict), accuracy_score(y_train, np.asarray(svc_clf.predict(X_train))))

0.5320056899004267 0.5327324478178368


C:\Users\14264\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [218]:
gaussNB = GaussianNB()
gaussNB.fit(X_train, y_train)
y_test_predict = gaussNB.predict(X_test)
print(accuracy_score(np.asarray(y_test), y_test_predict), accuracy_score(y_train, np.asarray(gaussNB.predict(X_train))))

0.5277382645803699 0.5426944971537002
